In [22]:
import requests
import json
import pandas as pd
from time import sleep

In [23]:
df = pd.read_csv('../output/codici_umbria_er_20241117.csv', dtype=str)
df_comuni = df[df['tipo'] == 'CM']
# Filtering for only Emilia Romagna
df_er = df_comuni[df_comuni['cod_regione'] == '08']

In [24]:
df_er.head(2)

,cod,desc,tipo,tipo_comune,dt_agg,cod_ele,cod_regione,regione,cod_provincia,provincia,cod_provincia_2,cod_comune
3,080130130005,ALTO RENO TERME,CM,N,NaN,7,08,EMILIA-ROMAGNA,013,BOLOGNA,013,0005
4,080130130010,ANZOLA DELL'EMILIA,CM,N,NaN,7,08,EMILIA-ROMAGNA,013,BOLOGNA,013,0010


In [25]:
BASE_URL = 'https://eleapi.interno.gov.it/siel/PX/getprefeR/DE/20241117/TE/07/RE/'

df_er['url_aff'] = df_er.apply(
    lambda row: f"{BASE_URL}{row['cod_regione']}/PR/{row['cod_provincia_2']}/CM/{row['cod_comune']}", axis=1
)

df_er['url'] = df_er.apply(
    lambda row: f"https://elezioni.interno.gov.it/risultati/20241117/regionali/scrutini/italia/{row['cod_regione']}{row['cod_provincia']}{row['cod_comune']}", axis=1
)
df_er.sample(3)

/var/folders/m0/g7ky_r7x67l1pv2zgg_fptx80000gn/T/ipykernel_93458/3249733948.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_er['url_aff'] = df_er.apply(
/var/folders/m0/g7ky_r7x67l1pv2zgg_fptx80000gn/T/ipykernel_93458/3249733948.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_er['url'] = df_er.apply(


,cod,desc,tipo,tipo_comune,dt_agg,cod_ele,cod_regione,regione,cod_provincia,provincia,cod_provincia_2,cod_comune,url_aff,url
289,080680680130,CASINA,CM,N,NaN,7,08,EMILIA-ROMAGNA,068,REGGIO NELL'EMILIA,068,0130,https://eleapi.interno.gov.it/siel/PX/getprefe...,https://elezioni.interno.gov.it/risultati/2024...
28,080130130280,GALLIERA,CM,N,NaN,7,08,EMILIA-ROMAGNA,013,BOLOGNA,013,0280,https://eleapi.interno.gov.it/siel/PX/getprefe...,https://elezioni.interno.gov.it/risultati/2024...
72,080290290130,MESOLA,CM,N,NaN,7,08,EMILIA-ROMAGNA,029,FERRARA,029,0130,https://eleapi.interno.gov.it/siel/PX/getprefe...,https://elezioni.interno.gov.it/risultati/2024...


In [26]:
headers = {
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'en-US,en;q=0.6',
    'dnt': '1',
    'origin': 'https://elezioni.interno.gov.it',
    'priority': 'u=1, i',
    'referer': 'https://elezioni.interno.gov.it/',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'sec-gpc': '1'
}

In [27]:
comuni_data = []

for idx, url in enumerate(df_er['url_aff']):
    print(f"\nProcessing record {idx + 1}")
    print(f"URL: {url}")

    response = requests.get(url, headers=headers)
    print(f"Response status code: {response.status_code}")
    
    data = response.json()

    comune = df_er.iloc[idx]['desc']
    provincia = df_er.iloc[idx]['provincia']
    regione = df_er.iloc[idx]['regione']
    url = df_er.iloc[idx]['url']

    print(f"Processing data for Comune: {comune}, Provincia: {provincia}")

    # Start with basic comune info
    comune_info = {
        'comune': comune,
        'provincia': provincia,
        'regione': regione,
        'url': url
    }
    
    # Group candidate votes by lista
    lista_votes = {}
    lista_names = {}

    # First, collect lista names and prepare vote tracking
    if 'liste' in data:
        for lista in data['liste']:
            lista_votes[lista['cod']] = 0
            lista_names[lista['cod']] = lista['desc']

    # Aggregate candidate votes by lista
    if 'cand' in data:
        for cand in data['cand']:
            lista_code = cand['cod_lis']
            lista_votes[lista_code] += cand['voti']

    # Add lista votes to comune_info
    for lista_code, votes in lista_votes.items():
        lista_name = lista_names.get(lista_code, f"Lista {lista_code}")
        comune_info[lista_name] = votes
        print(f"{lista_name}: {votes} votes")

    sezioni_scrutinate = data['liste'][0]['sez_perv']
    sezioni_tot = data['liste'][0]['sez_tot']

    # Optional: Verify consistency across lists
    if not all(lista.get('sez_perv') == sezioni_scrutinate for lista in data['liste']):
        print(f"Warning: Inconsistent scrutinized sections for {comune}")

    # Add to comune_info
    comune_info['sezioni_scrutinate'] = sezioni_scrutinate
    comune_info['sezioni_tot'] = sezioni_tot

    comuni_data.append(comune_info)
    
    print(f"Processed {comune}")
    sleep(0.1)

# Create DataFrame
print("\nCreating DataFrame...")
df_data = pd.DataFrame(comuni_data)

print("\nSample of processed data:")
print(df_data.head())


Processing record 1
URL: https://eleapi.interno.gov.it/siel/PX/getprefeR/DE/20241117/TE/07/RE/08/PR/013/CM/0005
Response status code: 200
Processing data for Comune: ALTO RENO TERME, Provincia: BOLOGNA
CIVICI, CON DE PASCALE PRESIDENTE: 0 votes
MOVIMENTO 5 STELLE: 0 votes
FRATELLI D'ITALIA - GIORGIA MELONI: 0 votes
LEGA SALVINI EMILIA-ROMAGNA - IL POPOLO DELLA FAMIGLIA: 0 votes
PARTITO DEMOCRATICO - DE PASCALE PRESIDENTE: 0 votes
RIFORMISTI EMILIA-ROMAGNA FUTURA - DE PASCALE PRESIDENTE: 0 votes
EMILIA-ROMAGNA PER LA PACE, L'AMBIENTE E IL LAVORO: 0 votes
RETE CIVICA - ELENA UGOLINI PRESIDENTE: 0 votes
LEALTA' COERENZA VERITA' - LUCA TEODORI: 0 votes
FORZA ITALIA - BERLUSCONI - UGOLINI PRESIDENTE - NOI MODERATI: 0 votes
ALLEANZA VERDI E SINISTRA - COALIZIONI CIVICHE - POSSIBILE: 0 votes
Processed ALTO RENO TERME

Processing record 2
URL: https://eleapi.interno.gov.it/siel/PX/getprefeR/DE/20241117/TE/07/RE/08/PR/013/CM/0010
Response status code: 200
Processing data for Comune: ANZOLA DEL

KeyboardInterrupt: 

In [ ]:
df_data

,comune,provincia,regione,url,"CIVICI, CON DE PASCALE PRESIDENTE",MOVIMENTO 5 STELLE,FRATELLI D'ITALIA - GIORGIA MELONI,LEGA SALVINI EMILIA-ROMAGNA - IL POPOLO DELLA FAMIGLIA,PARTITO DEMOCRATICO - DE PASCALE PRESIDENTE,RIFORMISTI EMILIA-ROMAGNA FUTURA - DE PASCALE PRESIDENTE,"EMILIA-ROMAGNA PER LA PACE, L'AMBIENTE E IL LAVORO",RETE CIVICA - ELENA UGOLINI PRESIDENTE,LEALTA' COERENZA VERITA' - LUCA TEODORI,FORZA ITALIA - BERLUSCONI - UGOLINI PRESIDENTE - NOI MODERATI,ALLEANZA VERDI E SINISTRA - COALIZIONI CIVICHE - POSSIBILE,sezioni_scrutinate,sezioni_tot
0,ALTO RENO TERME,BOLOGNA,EMILIA-ROMAGNA,https://elezioni.interno.gov.it/risultati/2024...,0,0,0,0,0,0,0,0,0.0,0,0,0,9
1,ANZOLA DELL'EMILIA,BOLOGNA,EMILIA-ROMAGNA,https://elezioni.interno.gov.it/risultati/2024...,0,0,0,0,0,0,0,0,0.0,0,0,0,11
2,ARGELATO,BOLOGNA,EMILIA-ROMAGNA,https://elezioni.interno.gov.it/risultati/2024...,0,0,0,0,0,0,0,0,0.0,0,0,0,9
3,BARICELLA,BOLOGNA,EMILIA-ROMAGNA,https://elezioni.interno.gov.it/risultati/2024...,0,0,0,0,0,0,0,0,0.0,0,0,0,7
4,BENTIVOGLIO,BOLOGNA,EMILIA-ROMAGNA,https://elezioni.interno.gov.it/risultati/2024...,0,0,0,0,0,0,0,0,0.0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,SANT'AGATA FELTRIA,RIMINI,EMILIA-ROMAGNA,https://elezioni.interno.gov.it/risultati/2024...,0,0,0,0,0,0,0,0,0.0,0,0,0,5
326,SANTARCANGELO DI ROMAGNA,RIMINI,EMILIA-ROMAGNA,https://elezioni.interno.gov.it/risultati/2024...,0,0,0,0,0,0,0,0,0.0,0,0,8,20
327,SASSOFELTRIO,RIMINI,EMILIA-ROMAGNA,https://elezioni.interno.gov.it/risultati/2024...,0,0,0,0,0,0,0,0,0.0,0,0,2,3
328,TALAMELLO,RIMINI,EMILIA-ROMAGNA,https://elezioni.interno.gov.it/risultati/2024...,0,0,0,0,0,0,0,0,0.0,0,0,1,1


In [ ]:
df_data['comune'] = df_data['comune'].str.title()
df_data['provincia'] = df_data['provincia'].str.title()
df_data['regione'] = df_data['regione'].str.title()
df_data

,comune,provincia,regione,url,"CIVICI, CON DE PASCALE PRESIDENTE",MOVIMENTO 5 STELLE,FRATELLI D'ITALIA - GIORGIA MELONI,LEGA SALVINI EMILIA-ROMAGNA - IL POPOLO DELLA FAMIGLIA,PARTITO DEMOCRATICO - DE PASCALE PRESIDENTE,RIFORMISTI EMILIA-ROMAGNA FUTURA - DE PASCALE PRESIDENTE,"EMILIA-ROMAGNA PER LA PACE, L'AMBIENTE E IL LAVORO",RETE CIVICA - ELENA UGOLINI PRESIDENTE,LEALTA' COERENZA VERITA' - LUCA TEODORI,FORZA ITALIA - BERLUSCONI - UGOLINI PRESIDENTE - NOI MODERATI,ALLEANZA VERDI E SINISTRA - COALIZIONI CIVICHE - POSSIBILE,sezioni_scrutinate,sezioni_tot
0,Alto Reno Terme,Bologna,Emilia-Romagna,https://elezioni.interno.gov.it/risultati/2024...,0,0,0,0,0,0,0,0,0.0,0,0,0,9
1,Anzola Dell'Emilia,Bologna,Emilia-Romagna,https://elezioni.interno.gov.it/risultati/2024...,0,0,0,0,0,0,0,0,0.0,0,0,0,11
2,Argelato,Bologna,Emilia-Romagna,https://elezioni.interno.gov.it/risultati/2024...,0,0,0,0,0,0,0,0,0.0,0,0,0,9
3,Baricella,Bologna,Emilia-Romagna,https://elezioni.interno.gov.it/risultati/2024...,0,0,0,0,0,0,0,0,0.0,0,0,0,7
4,Bentivoglio,Bologna,Emilia-Romagna,https://elezioni.interno.gov.it/risultati/2024...,0,0,0,0,0,0,0,0,0.0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,Sant'Agata Feltria,Rimini,Emilia-Romagna,https://elezioni.interno.gov.it/risultati/2024...,0,0,0,0,0,0,0,0,0.0,0,0,0,5
326,Santarcangelo Di Romagna,Rimini,Emilia-Romagna,https://elezioni.interno.gov.it/risultati/2024...,0,0,0,0,0,0,0,0,0.0,0,0,8,20
327,Sassofeltrio,Rimini,Emilia-Romagna,https://elezioni.interno.gov.it/risultati/2024...,0,0,0,0,0,0,0,0,0.0,0,0,2,3
328,Talamello,Rimini,Emilia-Romagna,https://elezioni.interno.gov.it/risultati/2024...,0,0,0,0,0,0,0,0,0.0,0,0,1,1


In [ ]:
df_data.to_csv('../output/risultati_er.csv', index=False, encoding='UTF-8-sig')